In [ ]:
import psycopg2
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

In [ ]:
"""This mocks up showing a preview of the top 10 segments that 
match the criteria, in this case recency. We could also use other
criteria such as segment length, average segment value, recency, etc.
"""

# Connect to DB
conn = psycopg2.connect("dbname=testdata user=postgres host=localhost password=postgres")
cur = conn.cursor()

# TODO: Change filter ranges and rerun processing to get new temp table

# Query across temp processed table to get top k segments by recency
# We should replace this with a new query once we've finished our implementation
# Note the json_agg function, which is used to return a json array of values across
# multiple rows. This can then be plotted directly in the UI.
query = """SELECT start_date, end_date, json_agg(sine1_value ORDER BY my_datetime)
FROM sinewaves_matching_ranges_temp, sinewaves_data
WHERE sinewaves_data.my_datetime >=start_date AND sinewaves_data.my_datetime <= end_date
GROUP BY start_date, end_date
ORDER BY end_date DESC
LIMIT 10"""

cur.execute(query)

start_end_dates = []
data = []

for record in cur:
    start_end_dates.append((record[0], record[1]))
    data.append(record[2])

# TODO: parameterize number of plots
gs = gridspec.GridSpec(10,1)
fig = plt.figure()
# Now plot data
for idx, recs in enumerate(data):
    ax = fig.add_subplot(gs[idx])
    ax.plot(recs)

In [ ]:
# Show a window for a specific plot
idx = 0  # 0 to 9

# Note the use of two json_agg functions to get the value and the 
# timestamps
# TODO: parameterize window
query = """SELECT json_agg(my_datetime ORDER BY my_datetime), 
json_agg(sine1_value ORDER BY my_datetime)
FROM sinewaves_data
WHERE my_datetime >= timestamp '2018-12-14 05:06:00' - interval '30 minutes' AND
my_datetime <= timestamp '2018-12-14 05:07:00' + interval '30 minutes';"""

cur.execute(query)

x_values = []
results = []
for record in cur:
    x_values.append(record[0])
    results.append(record[1])

plt.plot(x_values[0], results[0])